In [ ]:
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

### Pré processamento

In [ ]:
dataset = pd.read_csv('../data/raw/IMDB_Dataset.csv')

In [ ]:
dataset.shape

In [ ]:
dataset['sentiment'].value_counts()

In [ ]:
dataset['sentiment'] = dataset['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
dataset['review'] = dataset['review'].str.lower()

In [ ]:
#separando conjunto de treino e de teste
dados_treino, dados_teste = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
#transformando cada palavra de cada frase em números inteiros utilizando tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(dados_treino["review"])
x_treino = pad_sequences(tokenizer.texts_to_sequences(dados_treino["review"]), maxlen=200)
x_teste = pad_sequences(tokenizer.texts_to_sequences(dados_teste["review"]), maxlen=200)

In [ ]:
x_treino

In [ ]:
x_teste

In [ ]:
y_treino = dados_treino["sentiment"]
y_teste = dados_teste["sentiment"]

### Treinamento e validação

In [ ]:
##Arquitetura da rede neural: Camddas de dropout para evitar overfitting, Embedding para transformar palavras em
##vetores densos e LSTM para compreender o dependências e conntexto da sequência 
modelo = Sequential()
modelo.add(Embedding(input_dim=5000, output_dim=128))
modelo.add(Dropout(0.2))
modelo.add(LSTM(128, dropout=0.2, recurrent_dropout=0.1))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dropout(0.2))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))

In [ ]:
modelo.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
##early stopping para parar o treino quando o modelo parar de diminuir a função de perda do conjunto de validação
##learning rate dinâmico para ajudar na não estagnação da função de perda
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)

In [ ]:
##treino
modelo.fit(x_treino, y_treino, epochs=15, batch_size=64, validation_split=0.2, callbacks=[early_stop, reduce_lr])

In [ ]:
##avaliando o modelo no conjutno de validação 
loss, accuracy = modelo.evaluate(x_teste, y_teste)
print(f"Loss Teste: {loss}")
print(f"Accuracy Teste: {accuracy}")

### Usando o modelo

In [ ]:
##função para testar o modelo com suas próprias reviews ;)
###AVISO: O modelo foi treinado com avaliações feitas em INGLÊS, portanto escreva em inglês.
def predict_sentiment(review):
    review = review.lower() 
    sequencia = tokenizer.texts_to_sequences([review])
    padded_sequencia = pad_sequences(sequencia, maxlen=200)
    predicao = modelo.predict(padded_sequencia)
    sentiment = "positivo" if predicao[0][0] > 0.5 else "negativo"
    return sentiment

In [ ]:
minha_review = ""

sentiment = predict_sentiment(minha_review)
print(f'A review foi: {sentiment}')